This notebook was used to construct the examples in `07embedding/embedding_worksheet.docx`.

In [2]:
import torch
from torch import tensor
import pandas as pd

In [3]:
torch.manual_seed(0)
n_dim = 3
U = torch.randn(5, n_dim).round()
M = torch.randn(3, n_dim).round()


In [4]:
pd.DataFrame(M.numpy()).to_clipboard(header=False)

In [5]:
pd.Series([f'User {i+1}' for i in range(5)]).to_clipboard(index=False, header=False)

In [6]:
M

tensor([[ 0., -1.,  0.],
        [-1., -2.,  0.],
        [ 1.,  1.,  1.]])

In [7]:
pairs = [(1, 0), (0, 1), (3, 2)]
for uu, mm in pairs:
    print(f"dot(user {uu+1}, movie {mm+1}) = {U[uu] @ M[mm]}")

dot(user 2, movie 1) = 1.0
dot(user 1, movie 2) = -2.0
dot(user 4, movie 3) = -1.0


Constructing embeddings

Targets:

In [8]:
uu = torch.tensor([
    [1.0, -0.5],
    [0.0, 1.0],
])
vv = torch.tensor([
    [1.0, 1.0],
    [0.0, 1.0],
])
uu @ vv.T


tensor([[ 0.5000, -0.5000],
        [ 1.0000,  1.0000]])

In [9]:
ratings = torch.tensor([
  [1.0, 0.0, -1.0],
  [0.0, -1.0, 1.0]
])

In [10]:
# compute the PCA of ratings
# subtract the mean from each row
ratings_centered = ratings - ratings.mean(dim=-1, keepdim=True)
ratings_centered


tensor([[ 1.,  0., -1.],
        [ 0., -1.,  1.]])

In [11]:
# compute the SVD
svd_result = torch.svd(ratings_centered)
svd_result

torch.return_types.svd(
U=tensor([[ 0.7071,  0.7071],
        [-0.7071,  0.7071]]),
S=tensor([1.7321, 1.0000]),
V=tensor([[ 4.0825e-01,  7.0711e-01],
        [ 4.0825e-01, -7.0711e-01],
        [-8.1650e-01,  3.4572e-08]]))

In [12]:
U = torch.tensor([[1.0], [-1.0]])
V = torch.zeros(3, 1, requires_grad=True)

In [13]:
U[0] @ V[0]

tensor(0., grad_fn=<DotBackward0>)

In [14]:
loss = (U[0] @ V[0] - 1.0) ** 2
loss

tensor(1., grad_fn=<PowBackward0>)

In [15]:
loss.backward()
V.grad

tensor([[-2.],
        [ 0.],
        [ 0.]])

In [16]:
V.data -= 0.1 * V.grad

In [17]:
loss = (U[0] @ V[0] - 1.0) ** 2
loss

tensor(0.6400, grad_fn=<PowBackward0>)

In [18]:
V

tensor([[0.2000],
        [0.0000],
        [0.0000]], requires_grad=True)

In [23]:
def loss_and_m1_grad(u1, m1, target):
    m1 = torch.tensor(m1, requires_grad=True)
    pred = u1 * m1
    diff = target - pred
    loss = diff ** 2
    loss.backward()
    return loss.item(), m1.grad.item()

loss_and_m1_grad(1.0, 0.5, 1.0)

(0.25, -1.0)